In [ ]:
# Librairies
import pandas as pd
from ExtractionPipeline import extraction_pipeline
import tomllib

In [ ]:
df = pd.read_csv("../data/01_251021_AvisFiscalite.csv")
col_name = "QUXVlc3Rpb246MTYz - Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?"
df = df[['authorId', col_name]].rename({"authorId": "author_id", col_name:'contribution'}, axis=1)
df = df.iloc[:100]  # Limitation à 100 lignes pour les tests

In [ ]:
with open("01_251021_Prompt.toml", "rb") as f:
    toml_data = tomllib.load(f)

system_prompt = toml_data["prompt"]["system"]
user_template = toml_data["prompt"]["user"]

In [ ]:
dict_results = extraction_pipeline(
    df, 
    extract_model="gemma3:1b",
    system_prompt=system_prompt,
    user_template=user_template,
    return_sentiments=True,
    return_complet=True,
    error_filter=False,
    qualit_filter=0.0,
    rouge1_filter=0.0,
    rougeL_filter=0.0,
    nli_filter=0.0
)

In [ ]:
    # Calcul du score NLI
    device_t = torch.device(device)
    tokenizer = AutoTokenizer.from_pretrained(nli_model, use_fast=False)
    model = AutoModelForSequenceClassification.from_pretrained(nli_model).to(device_t)
    model.eval()
    cfg = AutoConfig.from_pretrained(nli_model)
    id2label = {int(k): v for k, v in cfg.id2label.items()} if isinstance(cfg.id2label, dict) else cfg.id2label
    entail_idx = next((k for k, v in id2label.items() if "entail" in str(v).lower()), 2)
    contra_idx = next((k for k, v in id2label.items() if "contrad" in str(v).lower()), 0)
    n = len(data_extracted)
    entail = np.zeros(n, dtype=np.float32)
    contra = np.zeros(n, dtype=np.float32)
    with torch.inference_mode():
        for start in range(0, n, 1):
            end = min(start + 1, n)
            premises = data_extracted["contribution"].iloc[start:end].fillna("").astype(str).tolist()
            hypos = data_extracted["extraction"].iloc[start:end].fillna("").astype(str).tolist()

            enc = tokenizer(
                premises,
                hypos,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt",
            )
            enc = {k: v.to(device_t) for k, v in enc.items()}
            probs = torch.softmax(model(**enc).logits, dim=1).detach().cpu().numpy()
            entail[start:end] = probs[:, entail_idx]
            contra[start:end] = probs[:, contra_idx]
    data_extracted["nli_entailment"] = entail.astype(float)
    data_extracted["nli_contradiction"] = contra.astype(float)
    data_extracted["nli_score"] = (1.0 - data_extracted["nli_contradiction"]).astype(float)